In [555]:
import torch
import pomdp
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [762]:
T, O, R = pomdp.env_simple(incorrect=0, coherence=1)
simple_pomdp_policy = pomdp.env_simple_fsc()
# some baselines... see what learning does?
spa, sps = pomdp.fsc_to_stochastic(simple_pomdp_policy[0], simple_pomdp_policy[1], T)
print(pomdp.policy_evaluation_sr(spa, sps, T, O, R, 0.95))
spa, sps = pomdp.random_fsc(2, T, O, R)
pomdp.policy_evaluation_sr(spa, sps, T, O, R, 0.95)

tensor([[20.0000, 19.0000],
        [19.0000, 20.0000]])


tensor([[ 9.8796, 10.1191],
        [ 9.8667, 10.1315]])

In [400]:
T = torch.tensor([
    # s = 0, a = 0,1,2
    [
        [0.5, 0.5],
        [0.5, 0.5],
        [1.0, 0.0],
    ],
    # s = 1, a = 0,1,2
    [
        [0.5, 0.5],
        [0.5, 0.5],
        [0.0, 1.0],
    ]
])

p = 0.85
O = torch.tensor([
    # a = 0, s' = 0,1
    [
        [0.5, 0.5],
        [0.5, 0.5],
    ],
    # a = 1, s' = 0,1
    [
        [0.5, 0.5],
        [0.5, 0.5],
    ],
    # a = 2, s' = 0,1
    [
        [p, 1-p],
        [1-p, p],
    ]
])

R = torch.tensor([
    # s = 0
    [+10, -100, -1],
    # s = 1
    [-100, +10, -1],
]).float()

In [401]:
opt_fsc_action = torch.tensor([2, 2, 0, 2, 1])
opt_fsc_state = torch.tensor([
    [1, 3],
    [2, 0],
    [0, 0],
    [0, 4],
    [0, 0],
])

In [350]:
gamma = 0.95

num_nodes, num_states = len(fsc_action), T.shape[0]
num_obs = O.shape[2]

V = torch.zeros((num_nodes, num_states))
for idx in range(200):
    prev = torch.clone(V)
    for n in range(num_nodes):
        for s in range(num_states):
            a = fsc_action[n]
            V[n, s] = R[s, a] + gamma * sum([
                T[s, a, ns] * sum(
                    O[a, ns, o] * V[fsc_state[n, o], ns]
                    for o in range(num_obs)
                )
                for ns in range(num_states)
            ])
    if torch.norm(prev-V) < 1e-5:
        print('Converged', idx)
        break
print(V)

IndexError: tensors used as indices must be long, byte or bool tensors

In [351]:
def policy_evaluation(fsc_action, fsc_state, T, O, R, *, gamma=0.95, debug=True, max_iter=5000, vi_eps=1e-3):
    num_nodes, num_states = len(fsc_action), T.shape[0]
    num_obs = O.shape[2]

    V = torch.zeros((num_nodes, num_states))
    for idx in range(max_iter):
        prev = torch.clone(V)
        for n in range(num_nodes):
            a = fsc_action[n]
            state_vals = torch.sum(O[a] * V[fsc_state[n]].T, axis=1)
            V[n] = R[:, a] + gamma * T[:, a] @ state_vals
        '''
        for n in range(num_nodes):
            for s in range(num_states):
                a = fsc_action[n]
                nn = fsc_state[n]

                state_vals = np.sum(O[a] * V[fsc_state[n]].T, axis=1)
                V[n, s] = R[s, a] + gamma * T[s, a] @ state_vals
        '''
        if torch.norm(prev-V) < vi_eps:
            if debug:
                print('Converged', idx)
            break
    return V
policy_evaluation(opt_fsc_action, opt_fsc_state, T, O, R)

Converged 108


tensor([[ 19.3677,  19.3677],
        [ 24.6921,   3.0113],
        [ 28.3993, -81.6007],
        [  3.0113,  24.6921],
        [-81.6007,  28.3993]])

# Stochastic controllers...

In [544]:
def det_to_stoch(fsc_action, fsc_state, T):
    num_states, num_actions, _ = T.shape
    num_nodes, num_obs = fsc_state.shape
    a = torch.zeros((num_nodes, num_actions))
    s = torch.zeros((num_nodes, num_actions, num_obs, num_nodes))
    for n in range(num_nodes):
        a[n, fsc_action[n]] = 1.
        for o in range(num_obs):
            s[n, fsc_action[n], o, fsc_state[n, o]] = 1.
    return a, s.sum(1)

det_to_stoch(opt_fsc_action, opt_fsc_state, T)

(tensor([[0., 0., 1.],
         [0., 0., 1.],
         [1., 0., 0.],
         [0., 0., 1.],
         [0., 1., 0.]]), tensor([[[0., 1., 0., 0., 0.],
          [0., 0., 0., 1., 0.]],
 
         [[0., 0., 1., 0., 0.],
          [1., 0., 0., 0., 0.]],
 
         [[1., 0., 0., 0., 0.],
          [1., 0., 0., 0., 0.]],
 
         [[1., 0., 0., 0., 0.],
          [0., 0., 0., 0., 1.]],
 
         [[1., 0., 0., 0., 0.],
          [1., 0., 0., 0., 0.]]]))

In [552]:
# http://www.ifaamas.org/Proceedings/aamas2015/aamas/p1249.pdf
# https://arxiv.org/pdf/1301.6720.pdf
def policy_evaluation_sr(fsc_action, fsc_state, T, O, R, *, gamma=0.95, dtype=torch.FloatTensor):
    num_states, num_actions, _ = T.shape
    num_nodes, num_obs = fsc_state.shape[0], O.shape[2]

    fsc_action = fsc_action.type(dtype)
    fsc_state = fsc_state.type(dtype)
    T = T.type(dtype)
    O = O.type(dtype)
    R = R.type(dtype)
        
    '''
    T # s, a, s' -> p(s'|s,a)
    O # a, s', o -> p(o|a,s')
    fsc_action # n,a -> p(a|n)
    fsc_state # n, a, o, n' -> p(a,n'|n,o) OR n, o, n' -> p(n'|n,o)

    p(n', s') = p(n'|s')p(s')
    p(s', a) = p(s'|a)p(a)
    p(n'|s') = 

    # given n,s
    p(n', a, s') = p(n'|s',a) p(s',a)

    p(a,s') = p(s'|a) p(a)

    p(a) = p(a|n), from FSC
    p(s'|a) = T(s, a, s') = p(s' | s, a), from transitions
    p(n'|s', a) = p(n'|n, o) p(o|a,s')
    
    '''
    if len(fsc_state.shape) == 4:
        fsc_state = fsc_state.sum(axis=1) # marginalizing over actoin, makes it p(n'|,n,o)

    assert torch.allclose(fsc_action.sum(-1), torch.ones(num_nodes).type(dtype))
    assert torch.allclose(fsc_state.sum(-1), torch.ones((num_nodes, num_obs)).type(dtype))
    assert torch.allclose(T.sum(-1), torch.ones((num_states, num_actions)).type(dtype))
    assert torch.allclose(O.sum(-1), torch.ones((num_actions, num_states)).type(dtype))

    # we want p(n',s'|n,s)
    Tmu = torch.zeros((num_nodes, num_states, num_nodes, num_states)).type(dtype)
    for n in range(num_nodes):
        for s in range(num_states):
            '''
            ns = T[s].T @ fsc_action[n]
            o = (O.T @ fsc_action[n]) @ ns
            nn = fsc_state[n].T @ o
            Tmu[n, s] = nn[:, None] * ns[None, :] # HACK HACK is this right?
            '''
            a_ns = T[s] * fsc_action[n, :, None]
            assert torch.allclose(a_ns.sum(), torch.ones(1).type(dtype))
            # (a, s', o) @ (o, n') = (a, s', n'), p(n'|a, s')
            p_nn = O @ fsc_state[n]
            assert torch.allclose(p_nn.sum(2), torch.ones(num_actions, num_states).type(dtype)), p_nn
            Tmu[n, s] = (a_ns[:, :, None] * p_nn).sum(0).T
            assert torch.allclose(Tmu[n, s].sum(), torch.ones(1).type(dtype))
            # hack bad
            #o = (O * a_ns[:, :, None]).sum((0, 1))
            #print(o.shape, fsc_state[n].T.shape)
            #nn = fsc_state[n].T @ o
            #Tmu[n, s] = nn[:, None] * ns[None, :] # HACK HACK is this right?

    # expected reward, (nodes, states)
    Cmu = fsc_action@R.T

    crossprod = num_nodes * num_states

    sr = (torch.eye(crossprod).type(dtype) - gamma * Tmu.view((crossprod, crossprod))).inverse()
    V = sr @ Cmu.view(crossprod)
    return V.view((num_nodes, num_states))

V = policy_evaluation_sr(*det_to_stoch(opt_fsc_action, opt_fsc_state, T) + (T, O, R))
print(V.shape, V)
V = policy_evaluation_sr(*det_to_stoch(opt_fsc_action, opt_fsc_state, T) + (T, O, R), dtype=torch.DoubleTensor)
print(V.shape, V)

torch.Size([5, 2]) tensor([[ 19.3714,  19.3714],
        [ 24.6957,   3.0148],
        [ 28.4028, -81.5972],
        [  3.0148,  24.6957],
        [-81.5972,  28.4028]])
torch.Size([5, 2]) tensor([[ 19.3714,  19.3714],
        [ 24.6957,   3.0148],
        [ 28.4028, -81.5972],
        [  3.0148,  24.6957],
        [-81.5972,  28.4028]], dtype=torch.float64)


In [553]:
def policy_evaluation_stoch(fsc_action, fsc_state, T, O, R, *, gamma=0.95, debug=True, max_iter=5000, vi_eps=1e-3):
    num_states, num_actions, _ = T.shape
    num_nodes, num_obs = fsc_state.shape[0], O.shape[2]

    if len(fsc_state.shape) == 4:
        fsc_state = fsc_state.sum(axis=1) # marginalizing over actoin, makes it p(n'|,n,o)

    V = torch.zeros((num_nodes, num_states))
    for idx in range(max_iter):
        prev = torch.clone(V)
        #'''
        for n in range(num_nodes):
            # (a, s')
            val = torch.sum(
                # This was computing when we were marginalizing out actions in this step
                # (a, s', o) * (a, None, o, n') = (a, s', o, n') -(sum(ax=2))> (a, s', n')
                #(O[:, :, :, None] * fsc_state[n, :, None]).sum(axis=2) *

                # (a, s', o) @ (o, n') = (a, s', n')
                O @ fsc_state[n] *
                V.T[None, :, :], axis=2)
            # (s, a) expected future value for state/action
            Q = torch.sum(T * val[None, :, :], axis=2)
            V[n] = (R + gamma * Q) @ fsc_action[n] # HAKKK 
            #V[n] = R @ fsc_action[n] + gamma * Q.sum(1) # HAKKK  # HAKKK  # HAKKK 
        #'''
        '''
        for n in range(num_nodes):
            # (a, s')
            val = torch.sum(
                # (a, s', o) @ (o, n') = (a, s', n')
                O @ fsc_state[n] *
                V.T[None, :, :], axis=2)
            # (s, a) expected future value for state/action
            Q = torch.sum(T * val[None, :, :], axis=2)
            V[n] = (R + gamma * Q) @ fsc_action[n]
        '''
        '''
        np = torch
        for n in range(num_nodes):
            # (a, s')
            val = np.sum(
                # (a, s', o) @ (o, n') = (a, s', n')
                O @ fsc_state[n] *
                V.T[None, :, :], axis=2)
            # (s, a) expected future value for state/action
            Q = np.sum(T * val[None, :, :], axis=2)
            V[n] = (R + gamma * Q) @ fsc_action[n]
        '''
        '''
        np = torch
        for n in range(num_nodes):
            for s in range(num_states):
                # (a, s')
                val = np.sum(
                    # (a, s', o) @ (o, n') = (a, s', n')
                    O @ fsc_state[n] *
                    V.T[None, :, :], axis=2)
                # (a) expected future value for actions
                val = np.sum(T[s] * val, axis=1)
                V[n, s] = fsc_action[n] @ (R[s] + gamma * val)
        '''
        '''
        for n in range(num_nodes):
            for s in range(num_states):
                V[n, s] = sum(
                    fsc_action[n, a] * (
                        R[s, a] + gamma * sum([
                            T[s, a, ns] * O[a, ns, :] @ fsc_state[n, :, :] @ V[:, ns]
                            for ns in range(num_states)
                        ])
                    )
                    for a in range(num_actions))
        '''
        '''
        for n in range(num_nodes):
            for s in range(num_states):
                V[n, s] = sum(fsc_action[n, a] * (R[s, a] + gamma * sum([
                    T[s, a, ns] * sum(
                        O[a, ns, o] * sum(fsc_state[n, o, nn] * V[nn, ns] for nn in range(num_nodes))
                        for o in range(num_obs)
                    )
                    for ns in range(num_states)
                ])) for a in range(num_actions))
        '''
        '''
        # deterministic
        for n in range(num_nodes):
            a = fsc_action[n]
            state_vals = np.sum(O[a] * V[fsc_state[n]].T, axis=1)
            V[n] = R[:, a] + gamma * T[:, a] @ state_vals
        '''
        '''
        for n in range(num_nodes):
            for s in range(num_states):
                a = fsc_action[n]
                nn = fsc_state[n]

                state_vals = np.sum(O[a] * V[fsc_state[n]].T, axis=1)
                V[n, s] = R[s, a] + gamma * T[s, a] @ state_vals
        '''
        if torch.norm(prev-V) < vi_eps:
            if debug:
                print('Converged', idx)
            break
    return V

# TODO, make SR !@ TODO TODO
V = policy_evaluation_stoch(*det_to_stoch(opt_fsc_action, opt_fsc_state, T) + (T, O, R))
print(V.shape, V)

Converged 108
torch.Size([5, 2]) tensor([[ 19.3677,  19.3677],
        [ 24.6921,   3.0113],
        [ 28.3993, -81.6007],
        [  3.0113,  24.6921],
        [-81.6007,  28.3993]])


In [719]:
np.random.seed(42)
T, O, R = pomdp.tiger()
fsc_action, fsc_state = random_fsc(7, T, O, R, action_condition=True)
print(pomdp.policy_evaluation_stoch_old(fsc_action, fsc_state, T, O, R, 0.95, vi_eps=1e-10))
print(pomdp.policy_evaluation_stoch(fsc_action, fsc_state, T, O, R, 0.95, vi_eps=1e-10))
print(pomdp.policy_evaluation_sr(fsc_action, fsc_state, T, O, R, 0.95))
'''
exact, for action_condition=True

exact, for action_condition=False
tensor([[-604.2139, -582.0386],
        [-587.4158, -603.8062],
        [-607.6362, -578.1958],
        [-578.3529, -596.7704]])

tensor([[-604.2104, -582.0352],
        [-587.4125, -603.8028],
        [-607.6329, -578.1927],
        [-578.3497, -596.7672]])
'''
None

Converged 158
tensor([[-597.9113, -579.3220],
        [-582.1718, -600.5173],
        [-601.5294, -574.9779],
        [-572.0983, -594.1543],
        [-580.1440, -605.3372],
        [-588.0622, -585.0547],
        [-583.5579, -588.9789]])
Converged 160
tensor([[-597.9112, -579.3219],
        [-582.1716, -600.5172],
        [-601.5293, -574.9778],
        [-572.0982, -594.1542],
        [-580.1439, -605.3372],
        [-588.0621, -585.0546],
        [-583.5577, -588.9788]])
tensor([[-597.9116, -579.3223],
        [-582.1720, -600.5177],
        [-601.5297, -574.9782],
        [-572.0986, -594.1545],
        [-580.1443, -605.3375],
        [-588.0625, -585.0551],
        [-583.5581, -588.9792]])


In [236]:
# trying to write the constraint out...
#ret = (T * (O * (canz@V).transpose(1, 2)).sum(2)[None, :, :]).sum(axis=(1, 2))
#R@c_a + gamma * ret

In [789]:
def make_constraints(n, V, T, O, R):
    num_states, num_actions, _ = T.shape
    num_nodes = V.shape[0]
    num_obs = O.shape[2]

    canz_shape = torch.Size((num_actions, num_obs, num_nodes))

    args = 1 + num_actions + canz_shape.numel()

    delta_slice = slice(0, 1)
    c_a_slice = slice(1, 1 + num_actions)
    canz_slice = slice(1 + num_actions, None)

    G = torch.zeros((V.shape[1] + num_actions + canz_shape.numel(), args))
    h = torch.cat((
        -V[n],
        torch.zeros(num_actions + canz_shape.numel()),
    ))

    # Last spots encode c >= 0 as -c <= 0
    for idx in range(num_actions + canz_shape.numel()):
        G[num_states+idx, delta_slice.stop+idx] = -1

    for s in range(num_states):
        G[s, delta_slice] = 1
        G[s, c_a_slice] = -R[s]
        canz_coef = G[s, canz_slice].view(canz_shape)

        # (a, s', o)
        #xx = T[s, :, :, None]*O
        for a in range(num_actions):
            for o in range(num_obs):
                for nn in range(num_nodes):
                    #c_a,n_z = p(n_z|a,n) p(a|n)
                    #p(n_z|a,n) = p(n_z|a) = p(n_z|o) p(o|a)
                    #p(o|a) xxxx
                    canz_coef[a, o, nn] = -gamma * (T[s, a, :] * O[a, :, o] * V[nn]).sum()

    # Q has to be PSD... so I've scaled it down considerably here
    Q = torch.eye(args) * 1e-4
    p = torch.zeros(args)
    # The default optimization is an argmin, so this lets us maximize the delta!
    p[delta_slice] = -1

    # HACK HACK in the future, write these so we don't need c_a?? to be simpler??
    A = torch.zeros((num_actions+1, args))
    b = torch.zeros(A.shape[0])

    # \sum_a c_a = 1
    A[num_actions, c_a_slice] = 1
    b[num_actions] = 1

    # for each a, \sum_a c_a_n_z = c_a
    for a in range(num_actions):
        A[a, c_a_slice][a] = -1
        A[a, canz_slice].view(canz_shape)[a] = 1

    return Q, p, G, h, A, b

Q, p, G, h, A, b = make_constraints(0, V, T, O, R)
#A, b
G[2], h[2]

RuntimeError: Expected object of scalar type Double but got scalar type Float for sequence element 1 in sequence argument at position #1 'tensors'

In [548]:
import qpth
constraints = make_constraints(0, V, T, O, R)
soln = qpth.qp.QPFunction(verbose=-1)(*constraints).float()


In [549]:
soln.flatten()[canz_slice].view(canz_shape).sum()

tensor(nan)

In [389]:
[
    soln.flatten()[delta_slice],
    soln.flatten()[c_a_slice],
    soln.flatten()[canz_slice].view(canz_shape),
]

RuntimeError: shape '[3, 2, 3]' is invalid for input of size 30

In [390]:
fsc_action[n], fsc_state[n], soln.flatten()[canz_slice].view(canz.shape)[0, 1].sum()

(tensor([0.3333, 0.3333, 0.3333]), tensor([[[0.1250, 0.1250, 0.1250],
          [0.1250, 0.1250, 0.1250]],
 
         [[0.1250, 0.1250, 0.1250],
          [0.1250, 0.1250, 0.1250]],
 
         [[0.1250, 0.1250, 0.1250],
          [0.1250, 0.1250, 0.1250]]]), tensor(-9.8944))

# Trying to fit...

In [787]:
# grad works...
env = pomdp.env_simple(incorrect=0, coherence=1) # very simple
env = pomdp.env_simple()
d = pomdp.policy_iteration_grad(env, 0.95, grad_steps=200, progress=25)
log_fsc(d['fsc_action'], d['fsc_state'])

24 -14.195177063973624
49 -15.048375633298345
74 -15.10550803890106
99 -15.127054119649422
124 -15.14114085893618
149 -15.151401423107314
174 -15.159131071502808
199 -15.165094320747649
action, obs, node
node 0 action p(a=0) 1.0
p(n=0|o=0) 0.999
p(n=1|o=1) 0.998
node 1 action p(a=1) 1.0
p(n=0|o=0) 0.999
p(n=1|o=1) 0.999


In [776]:
def log_fsc(fsc_action, fsc_state):
    print('action, obs, node')
    prev = -1
    for idx in zip(*[x.detach().numpy() for x in torch.where(fsc_state>.01)]):
        if prev != idx[0]:
            print('node', idx[0], 'action', ', '.join([
                f'p(a={i}) {round(p, 3)}'
                for i, p in enumerate(fsc_action[idx[0]].detach().numpy())
                if p > 1e-2]))
        if len(idx) == 3:
            o, n = idx[1:]
            msg = f'p(n={n}|o={o})'
        else:
            a, o, n = idx[1:]
            msg = f'p(n={n}|a={a},o={o})'
        print(msg, round(fsc_state[idx].item(), 3))
        prev = idx[0]
log_fsc(fsc_action, fsc_state)

action, obs, node
node 0 action p(a=0) 1.0
p(n=1|a=0,o=0) 1.0
p(n=0|a=0,o=1) 0.49
p(n=1|a=0,o=1) 0.51
p(n=0|a=1,o=0) 0.657
p(n=1|a=1,o=0) 0.343
p(n=0|a=1,o=1) 0.502
p(n=1|a=1,o=1) 0.498
node 1 action p(a=0) 0.85, p(a=1) 0.15
p(n=0|a=0,o=0) 1.0
p(n=0|a=0,o=1) 0.488
p(n=1|a=0,o=1) 0.512
p(n=0|a=1,o=0) 1.0
p(n=0|a=1,o=1) 0.487
p(n=1|a=1,o=1) 0.513


In [802]:
Q, p, G, h, A, b = pomdp.make_constraints_simple(n, V, T, O, R, gamma, dtype=dtype, qcoef=1e-10)[0]
print('Q', Q)
print('p', p)

print()
print('G', G)
print('h', h)

print()
print('A', A)
print('b', b)

Q tensor([[1.0000e-10, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00],
        [0.0000e+00, 1.0000e-10, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00, 1.0000e-10, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00, 0.0000e+00, 1.0000e-10, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 1.0000e-10, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 1.0000e-10,
         0.0000e+00, 0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         1.0000e-10, 0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 1.0000e

In [799]:
import pomdp
T, O, R = pomdp.env_simple(incorrect=0, coherence=1)

num_states, num_actions, _ = T.shape
num_obs = O.shape[2]

# canz -> p(n' | n, a, o) -> p(n', a, o | n) ??
# c_a -> p(a | n)
num_nodes = 5
fsc_action = torch.ones((num_nodes, num_actions))/num_actions
#fsc_state = torch.ones((num_nodes, num_actions, num_obs, num_nodes))/(num_actions+num_obs+num_nodes)
#fsc_state = torch.ones((num_nodes, num_obs, num_nodes))/(num_nodes)
fsc_state = torch.ones((num_nodes, num_actions, num_obs, num_nodes))/(num_actions+num_obs+num_nodes)

num_nodes = 2

fsc_action, fsc_state = pomdp.random_fsc(num_nodes, T, O, R, dtype=dtype, action_condition=True)
#fsc_action, fsc_state = det_to_stoch(opt_fsc_action, opt_fsc_state, T)

#fsc_action = torch.tensor(np.random.uniform(0, 1, size=(num_nodes, num_actions))).softmax(1).float()
#xrnd = np.random.uniform(0, 1, size=(num_nodes, num_actions, num_obs, num_nodes))
#fsc_state = torch.tensor(xrnd/np.sum(xrnd, axis=(1, 2, 3))).float()

canz_shape = torch.Size((num_actions, num_obs, num_nodes))

debug=True

gamma = 0.95
dtype = torch.float64
steps = 2

V = pomdp.policy_evaluation_sr(fsc_action, fsc_state, T, O, R, gamma, dtype=dtype)
#V = pomdp.policy_evaluation_stoch(fsc_action, fsc_state, T, O, R, debug=False)
for i in range(steps):
    prev = V.clone()
    for n in range(num_nodes):
        V = pomdp.policy_evaluation_sr(fsc_action, fsc_state, T, O, R, gamma, dtype=dtype)
        if debug:
            print('value', V)
        #V = policy_evaluation_stoch(fsc_action, fsc_state, T, O, R, debug=False)
        constraints, unpack = pomdp.make_constraints_simple(n, V, T, O, R, gamma, dtype=dtype, qcoef=1e-10)
        #soln = qpth.qp.QPFunction(verbose=True, solver=qpth.qp.QPSolvers.CVXPY)(*constraints).float()
        soln = qpth.qp.QPFunction(verbose=debug)(*constraints).float()
        #delta = soln[0, 0].item()
        delta, c_a, canz = unpack(soln[0])
        print('node', n, 'V delta', delta)
        if debug:
            print('node', n, 'V delta', delta)
            print('sums', c_a.sum(), canz.sum(), 'c_a', c_a)
        if delta > 0: # HACK check tos ee if xxx
            #fsc_action[n] = soln[0, c_a_slice]
            #fsc_action[n] = c_a
            # normalize!!??
            #fsc_action[n] = fsc_action[n] / fsc_action[n].sum()
            #do we marginalize over actions?
            # do we normalize so this is P(n'|n,o,a), instead of p(a,n'|n,o)?
            #canz = soln[0, canz_slice].view(canz_shape)
            # normalizing by p(a|n) to give us p(n'|n,o)
            #fsc_state[n] = canz/canz.sum(0)
            # HACK actually let's marginalize over actions first then normalize
            # HACK actually we need to change this per the later meuleau
            #canz = canz.sum(0)
            #print(canz, canz.shape, canz.sum(0).shape, fsc_state[n].shape)
            #print(canz, canz.shape, canz.sum(1), (canz/canz.sum(1)[None, :]).sum(1), c_a)
            #fsc_state[n] = canz/canz.sum(1)[:, None]

            # NEW
            fsc_action[n] = c_a
            fsc_state[n] = canz/canz.sum(-1)[:, :, None]

    V = pomdp.policy_evaluation_sr(fsc_action, fsc_state, T, O, R, gamma, dtype=dtype)
    print(i, V, (prev-V).norm())
    if (prev-V).norm() < 1e-3:
        print('converged!')
        #break

value tensor([[10.0065, 10.2781],
        [10.1721, 10.1018]], dtype=torch.float64)
iter: 0, pri_resid: 4.75180e+00, dual_resid: 3.86813e+01, mu: 1.69933e+00
iter: 1, pri_resid: 6.96482e-04, dual_resid: 5.80889e-15, mu: 1.38590e-01
iter: 2, pri_resid: 9.71255e-05, dual_resid: 2.36667e-15, mu: 6.13253e-03
iter: 3, pri_resid: 3.58792e-05, dual_resid: 2.35136e-15, mu: 3.66549e-03
iter: 4, pri_resid: 1.55608e-06, dual_resid: 2.04473e-15, mu: 7.74429e-05
iter: 5, pri_resid: 2.37925e-08, dual_resid: 1.39287e-15, mu: 8.64712e-08
iter: 6, pri_resid: 3.73501e-11, dual_resid: 1.28826e-15, mu: 8.64712e-11
iter: 7, pri_resid: 6.16878e-14, dual_resid: 1.72283e-15, mu: 8.64712e-14
node 0 V delta -4.781377792358398
node 0 V delta -4.781377792358398
sums tensor(1.) tensor(1.) c_a tensor([1.0000e+00, 2.4968e-11])
value tensor([[10.0065, 10.2781],
        [10.1721, 10.1018]], dtype=torch.float64)
iter: 0, pri_resid: 4.75396e+00, dual_resid: 3.85364e+01, mu: 1.69356e+00
iter: 1, pri_resid: 8.43227e-04, d

In [685]:
hi = fsc_state.shape
print(fsc_state.view(-1).repeat(2).view((2,)+hi))
print(fsc_state.view(-1).repeat(2).view((2,)+hi).transpose(1, 2))

tensor([[[[0.5000, 0.5000],
          [0.5002, 0.4998]],

         [[0.5000, 0.5000],
          [0.5003, 0.4997]]],


        [[[0.5000, 0.5000],
          [0.5002, 0.4998]],

         [[0.5000, 0.5000],
          [0.5003, 0.4997]]]])
tensor([[[[0.5000, 0.5000],
          [0.5000, 0.5000]],

         [[0.5002, 0.4998],
          [0.5003, 0.4997]]],


        [[[0.5000, 0.5000],
          [0.5000, 0.5000]],

         [[0.5002, 0.4998],
          [0.5003, 0.4997]]]])


In [681]:
fsc_action.view(-1).repeat(2).view()

tensor([2.9204e-15, 1.0000e+00, 2.9368e-15, 1.0000e+00, 2.9204e-15, 1.0000e+00,
        2.9368e-15, 1.0000e+00])